In [2]:
#!pip install numpy==1.22
#!pip install pandas==1.3.4
!pip install keras==2.12.0
!pip install daal==2021.4.0

  Using cached daal-2021.4.0-py2.py3-none-manylinux1_x86_64.whl (295.5 MB)
  Using cached tbb-2021.9.0-py2.py3-none-manylinux1_x86_64.whl (4.0 MB)
  Attempting uninstall: tbb
    Found existing installation: TBB 0.2
ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

2023-05-01 18:12:35.171493: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 18:12:35.650791: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 18:12:35.653209: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 18:12:38.447472: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
#download the data and read it into a pandas dataframe.
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [5]:
concrete_data.shape
concrete_data.describe()
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [6]:
#Split data into predictors and target

concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

predictors.head()
#predictors.shape
#target.shape

n_cols = predictors.shape[1] # number of predictors
n_cols


8

## Normalize Data



The next step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [37]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
target_norm = (target - target.mean()) / target.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [25]:
from keras import Sequential
from tensorflow.keras import layers
from keras.layers import Dense

In [26]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    #model.add(Dense(5, activation='relu'))
    #model.add(Dense(5, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

1)Randomly split the data into a training and test sets by holding 30% of the data for testing. 

In [38]:
from sklearn.model_selection import train_test_split

predictors_train, predictors_test, target_train, target_test = train_test_split(predictors_norm, target_norm, test_size=0.30,random_state=42)

#predictors_train.shape
target_test.shape
np.array(target_test).reshape(103, 3)#use reshape 103*3 = 309 original size
                                      #for the arg 'multiouput' in mean_squared_error function


array([[ 1.0231236 ,  1.20210398,  2.31549361],
       [-0.03100498, -1.51313011,  0.50653475],
       [-0.72597561,  0.57178178,  0.0947003 ],
       [ 0.78009339, -0.64157349, -1.50175677],
       [ 0.26230735,  0.83696006, -0.16808359],
       [-0.8319272 ,  0.25392699, -0.98576652],
       [ 0.08093258, -0.16329482, -0.26565484],
       [ 0.12822171,  0.77231163, -1.73101926],
       [ 0.16653189, -0.29738046, -1.47541853],
       [ 0.31318806,  1.02611658, -1.35270623],
       [ 0.98660921,  0.22639155,  0.96027096],
       [ 1.83182758, -1.24974761, -0.167485  ],
       [-0.25009133,  0.21561681, -1.32636798],
       [ 0.96625692, -1.21143743, -1.76813225],
       [ 0.13480627,  0.86329831, -1.54365854],
       [ 1.65105141,  1.1811531 ,  0.01987573],
       [-0.84210334, -1.85492876,  1.28052013],
       [ 0.71604355, -0.38357836, -1.40358694],
       [ 1.6881644 , -0.08487867, -0.62541138],
       [-1.32337499, -0.24530255, -0.96062547],
       [ 0.21561681, -1.33474833, -0.028

## Train and Test the Network


In [39]:
#Build the model

model = regression_model()


2)  Train the model on the training data using 50 epochs

In [47]:
# train the model with fit
model.fit(predictors_train, target_train, validation_split=0.3, epochs=100,verbose=2)

Epoch 1/100
16/16 - 0s - loss: 0.0920 - val_loss: 0.1396 - 165ms/epoch - 10ms/step
Epoch 2/100
16/16 - 0s - loss: 0.0913 - val_loss: 0.1401 - 108ms/epoch - 7ms/step
Epoch 3/100
16/16 - 0s - loss: 0.0915 - val_loss: 0.1411 - 106ms/epoch - 7ms/step
Epoch 4/100
16/16 - 0s - loss: 0.0915 - val_loss: 0.1407 - 109ms/epoch - 7ms/step
Epoch 5/100
16/16 - 0s - loss: 0.0915 - val_loss: 0.1386 - 112ms/epoch - 7ms/step
Epoch 6/100
16/16 - 0s - loss: 0.0916 - val_loss: 0.1409 - 109ms/epoch - 7ms/step
Epoch 7/100
16/16 - 0s - loss: 0.0916 - val_loss: 0.1406 - 106ms/epoch - 7ms/step
Epoch 8/100
16/16 - 0s - loss: 0.0916 - val_loss: 0.1398 - 103ms/epoch - 6ms/step
Epoch 9/100
16/16 - 0s - loss: 0.0916 - val_loss: 0.1416 - 106ms/epoch - 7ms/step
Epoch 10/100
16/16 - 0s - loss: 0.0915 - val_loss: 0.1394 - 111ms/epoch - 7ms/step
Epoch 11/100
16/16 - 0s - loss: 0.0914 - val_loss: 0.1412 - 103ms/epoch - 6ms/step
Epoch 12/100
16/16 - 0s - loss: 0.0915 - val_loss: 0.1403 - 102ms/epoch - 6ms/step
Epoch 13/100

In [48]:
#Predictions

predictions = model.predict(predictors_test)
np.array(predictions).reshape(103, 3) #use reshape 103*3 = 309 original size
                                      #for the arg 'multiouput' in mean_squared_error function


10/10 [==============================] - 0s 3ms/step


array([[ 5.40846586e-01,  6.16075277e-01,  1.89464378e+00],
       [ 4.14939553e-01, -1.34755301e+00,  5.73821068e-01],
       [-5.28192282e-01,  8.72725248e-01,  7.95072019e-02],
       [ 4.38150793e-01,  2.00073570e-02, -1.66153860e+00],
       [ 4.25311297e-01,  1.32706314e-01, -7.57377625e-01],
       [-7.17342138e-01,  1.85804129e-01, -1.14484644e+00],
       [ 5.37965000e-02, -3.45623285e-01, -1.48554951e-01],
       [ 2.59464115e-01,  6.34497166e-01, -1.20674562e+00],
       [-4.87641871e-01,  3.35006446e-01, -1.63514352e+00],
       [ 8.96505117e-01,  8.82545233e-01, -1.11038113e+00],
       [ 2.81762093e-01,  1.69363022e-01,  8.79157424e-01],
       [ 9.61979270e-01, -8.81240010e-01,  1.90727502e-01],
       [ 9.67440903e-02,  4.07972842e-01, -1.48903024e+00],
       [ 1.22071517e+00, -1.30625463e+00, -1.83137298e+00],
       [-4.32087034e-01,  7.22528219e-01, -1.36399090e+00],
       [ 2.14681387e+00,  1.20507210e-01,  6.57528639e-04],
       [-4.67465192e-01, -1.39184272e+00

3) Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.


In [49]:
from sklearn.metrics import mean_squared_error

In [50]:
metrics = mean_squared_error(target_test, predictions, multioutput = 'raw_values')
metrics

array([0.16974793])

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [52]:
metrics1 = np.zeros(50) #Incializando array
for i in range(49):
    model.fit(predictors_train, target_train, validation_split=0.3, epochs=100,verbose=2)
    predictions = model.predict(predictors_test)
    metrics1[i] = np.array(mean_squared_error(target_test, predictions, multioutput = 'raw_values'))
    print(metrics1[i])

Epoch 1/100
16/16 - 0s - loss: 0.0907 - val_loss: 0.1427 - 129ms/epoch - 8ms/step
Epoch 2/100
16/16 - 0s - loss: 0.0909 - val_loss: 0.1413 - 135ms/epoch - 8ms/step
Epoch 3/100
16/16 - 0s - loss: 0.0903 - val_loss: 0.1426 - 143ms/epoch - 9ms/step
Epoch 4/100
16/16 - 0s - loss: 0.0910 - val_loss: 0.1431 - 127ms/epoch - 8ms/step
Epoch 5/100
16/16 - 0s - loss: 0.0898 - val_loss: 0.1414 - 109ms/epoch - 7ms/step
Epoch 6/100
16/16 - 0s - loss: 0.0901 - val_loss: 0.1419 - 111ms/epoch - 7ms/step
Epoch 7/100
16/16 - 0s - loss: 0.0915 - val_loss: 0.1440 - 112ms/epoch - 7ms/step
Epoch 8/100
16/16 - 0s - loss: 0.0906 - val_loss: 0.1414 - 112ms/epoch - 7ms/step
Epoch 9/100
16/16 - 0s - loss: 0.0914 - val_loss: 0.1419 - 115ms/epoch - 7ms/step
Epoch 10/100
16/16 - 0s - loss: 0.0904 - val_loss: 0.1429 - 108ms/epoch - 7ms/step
Epoch 11/100
16/16 - 0s - loss: 0.0902 - val_loss: 0.1423 - 121ms/epoch - 8ms/step
Epoch 12/100
16/16 - 0s - loss: 0.0900 - val_loss: 0.1415 - 133ms/epoch - 8ms/step
Epoch 13/100


In [53]:
#array with 
metrics1

array([0.17743803, 0.17750239, 0.17767006, 0.17702655, 0.17771727,
       0.17740586, 0.17699917, 0.17621416, 0.17731263, 0.17799022,
       0.17716189, 0.17742532, 0.17651434, 0.17625924, 0.1767526 ,
       0.17705068, 0.17655079, 0.1775005 , 0.17898285, 0.17761174,
       0.17720337, 0.17674784, 0.1767491 , 0.17763935, 0.1785387 ,
       0.17732283, 0.17789375, 0.17665885, 0.17722433, 0.17745274,
       0.17605192, 0.17796398, 0.17848805, 0.17709717, 0.17872782,
       0.17725755, 0.17762289, 0.17630339, 0.17820042, 0.1775264 ,
       0.17791383, 0.17685687, 0.1761254 , 0.17744202, 0.17758789,
       0.17634051, 0.17693957, 0.17811984, 0.17763582, 0.        ])

5. Report the mean and the standard deviation of the mean squared errors.

In [54]:
#The std and mean values

from statistics import mean

metrics1_avg = mean(metrics1)
metrics1_std = np.std(metrics1)

print("The mean of the MSN array is: ", metrics1_avg, " and the standard deviation is: ", metrics1_std)


The mean of the MSN array is:  0.17377436994539885  and the standard deviation is:  0.02483385611230535
